In [2]:
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip
import speech_recognition as sr
import os
import tkinter
from tkinter.filedialog import askopenfilename
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

import json

In [14]:
# Extraigo el audio del vídeo

video = mp.VideoFileClip(".\\videos\\Borja_7.mp4")
audio = video.audio.write_audiofile(r".\\audios\\Borja_1.wav") # de cuando se recibia un vídeo
recibido_cambio = sr.Recognizer()
speech_audio = sr.AudioFile(".\\audios\\Borja_1.wav")

with speech_audio as fuente:
    #Se reduce el ruido
    recibido_cambio.adjust_for_ambient_noise(fuente)
    audio = recibido_cambio.record(fuente)
    #Transcripcion
    texto_transcrito = recibido_cambio.recognize_google(audio, language = "es-ES")
    print(texto_transcrito)

chunk:   0%|          | 0/1023 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in .\\audios\\Borja_1.wav


MoviePy - Done.
tengo la próxima rotonda rotonda media levanto el pie del acelerador lo tonta cerca y en la ocupada


In [4]:
# Guardo el texto en bruto transcrito de 

filename_t_raw = ".\\t_raw\\Borja_1.txt"
with open(filename_t_raw, "w", encoding="utf-8") as f:
    f.write(texto_transcrito)
print("Se ha guardado la transcripción en el archivo " + filename_t_raw)

Se ha guardado la transcripción en el archivo .\t_raw\Borja_1.txt
